### Problem Description

In this problem, a saloon owner wants to determine the schedule for staff members. The staff consists of the full-time shift of 9 hours and part-time shift of 3 hours.  The saloon’s opening hours are divided into 4 shifts of 3 hours each. In each shift, different levels of demands are there that need the different number of staff members in each shift.

The required number of hair dressers for each shift is mentioned in the below table:

|  Shift  ||    Time Period   ||   Number of Employees  |
| :---:        ||    :----:   ||      :---: |
|  Morning  ||    09AM-12PM  ||  6  |
|  Afternoon  ||    12-03PM  ||   11  |
|  Evening  || 3-06PM   ||   8  |
|  Night  ||    3-09PM   ||   6  |

There is at least 1 full-time employee we need in each shift. The full-time employee will get 150 EUR for 9 hours shift and the part-time employee will get 45 EURO per shift.

## Model Formulation
Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization problem that captures the key features of a complex decision problem. The Gurobi Optimizer solves the mathematical optimization problem using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components:
- Sets.
- Parameters.
- Decision variables.
- Constraints.
- Objective function(s).

### Sets and Indices

$s \in shifts$ = {Morning, Afternoon, Evening, Night}: Index and set of shifts.

$w \in workers= {Fulltime, Partime}$: Index and set of employed workers either Part-time or Fulltime.

$availability= \{(w,s): w \in workers \; \text{is available for} \; s \in \text{shifts} \}$: This set determines that the employed worker  $w$ is available to work on shift $s$.

### Parameters

$\text{shiftRequirements}(s) \in \mathbb{N}$: This parameter represents the number of workers required at each shift 
$s \in \text{shifts}$.

$pay(w) \in \mathbb{R^{+}}$: This parameter is the salary per day of each worker type $w \in workers$. 


### Decision Variables

$x(s)$: Number of full-time employees scheduled in shift $s \in \text{shifts}$, if the employee is Full-time.

$y(s)$: Number of part-time employees scheduled in shift $s \in \text{shifts}$ , if the employee is Part-time.


## Optimization Process

In this modeling example, the objective is to minimize the cost of  workers required to satisfy shift requirements.

- **objective:** The objective function is to minimize the cost of  workers required to satisfy shift requirements.
 
\begin{equation}
\text{Min} \quad 
\sum_{w \: \in \: workers} pay(w)*x(s)  + \sum_{w \: \in \: workers} pay(w)*y(s) =  \quad \forall \; s \in \text{shifts}
\tag{0}
\end{equation}

### Constraints

- **Shift requirements:**  Employee starting shift constraint requirements must be satisfied.

\begin{equation}
\sum_{w \: \in \: workers} x(s)  +  y(s) = \text{shiftRequirements}(s) \quad \forall \; s \in shifts = {Morning}
\tag{1}
\end{equation}

\begin{equation}
\sum_{w \: \in \: workers} x(s)  +  y(s) = \text{shiftRequirements}(s) \quad \forall \; s \in shifts = {Morning, Afternoon } 
\tag{2}
\end{equation}

\begin{equation}
\sum_{w \: \in \: workers} x(s)  +  x(s) +  y(s)  = \text{shiftRequirements}(s) \quad \forall \; s \in shifts = {Afternoon, evening } 
\tag{3}
\end{equation}

\begin{equation}
\sum_{w \: \in \: workers} x(s)  +  x(s) +  y(s)  = \text{shiftRequirements}(s) \quad \forall \; s \in shifts = {Afternoon, night } 
\tag{4}
\end{equation}



- **Shifts:** Minimum full-time employees during any shift/period.

\begin{equation}
\sum_{w \: \in \: \text{workers}} x(s)  = \text{shiftRequiremnts}(w) \geq 1 \quad \forall \; w \in workers
\tag{3}
\end{equation}

We now solve the following model where we minimize the  objective.


In [1]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import pandas as pd

In [2]:
#Parameters            
shifts = list(range(4))

In [3]:
model = gp.Model("staff_scheduling")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-05


In [4]:
#Initialize assignment decision variables.
x = model.addVars(shifts, vtype=GRB.INTEGER, name="fulltime")
y = model.addVars(shifts, vtype=GRB.INTEGER, name="part_time")

In [5]:
#Objective
model.setObjective(gp.quicksum(150*x[s]  + 45*y[s] 
                    for s in shifts), GRB.MINIMIZE )


In [6]:
# Constraint 1: morning shift requirements
morning_shift = model.addConstrs(
    (gp.quicksum(x[i] + y[j] for i in shifts if i == 0 ) >= 6 
    for j in shifts if j == 0 ), name="Morning")


# Constraint 2: afternoon shift requirements
Afternoon_shift = model.addConstrs(
    (gp.quicksum(x[m]  + x[a] + y[a] for m in shifts if m == 0 )  >= 8 
    for a in shifts if a == 1), name="Afternoon")

# Constraint 3: afternoon shift requirements
Evening_shift = model.addConstrs(
    (gp.quicksum(x[a]  + x[e] + y[e] for a in shifts if a == 1 )  >= 11
     for e in shifts if e == 2), name="Evening")

# Constraint 4: afternoon shift requirements
Night_shift = model.addConstrs(
    (gp.quicksum(x[e]  + x[n] + y[n] for e in shifts if e == 2 )  >= 6
     for n in shifts if n == 3 
     ), name="Night")

#Constraint 5:Minimum full-time employees during any shift/period

Fulltime_morning = model.addConstrs(
    (x[f] >= 1 for f in shifts if f == 0 ), name="fulltime_morning" )

Fulltime_afternoon = model.addConstrs(
    (x[f] >= 1 for f in shifts if f == 1 ), name="fulltime_afternoon" )

Fulltime_evening = model.addConstrs(
    (x[f] >= 1 for f in shifts if f == 2 ), name="fulltime_afternoon" )

In [7]:
#solve
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 7 rows, 8 columns and 14 nonzeros
Model fingerprint: 0x14b4f9a7
Variable types: 0 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1575.0000000
Presolve removed 3 rows and 1 columns
Presolve time: 0.10s
Presolved: 4 rows, 7 columns, 10 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.03 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.67 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 1575 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.575000000000e+03, best bound 1.575000000000e+03, gap 0.0000%


In [8]:
#Display model
model.display()

Minimize
<gurobi.LinExpr: 150.0 fulltime[0] + 150.0 fulltime[1] + 150.0 fulltime[2]
+ 150.0 fulltime[3] + 45.0 part_time[0] + 45.0 part_time[1] + 45.0 part_time[2]
+ 45.0 part_time[3]>
Subject To
  Morning[0]: <gurobi.LinExpr: fulltime[0] + part_time[0]> >= 6
  Afternoon[1]: <gurobi.LinExpr: fulltime[0] + fulltime[1] + part_time[1]> >= 8
  Evening[2]: <gurobi.LinExpr: fulltime[1] + fulltime[2] + part_time[2]> >= 11
  Night[3]: <gurobi.LinExpr: fulltime[2] + fulltime[3] + part_time[3]> >= 6
  fulltime_morning[0]: <gurobi.LinExpr: fulltime[0]> >= 1
  fulltime_afternoon[1]: <gurobi.LinExpr: fulltime[1]> >= 1
  fulltime_afternoon[2]: <gurobi.LinExpr: fulltime[2]> >= 1
General Integers
['fulltime[0]', 'fulltime[1]', 'fulltime[2]', 'fulltime[3]', 'part_time[0]',
 'part_time[1]', 'part_time[2]', 'part_time[3]']


In [11]:

#Output report

print(f"Total cost of Employee assigned: {round(model.objVal)} ") 



for v in model.getVars():
    print(v.varName,v.x)



Total cost of Employee assigned: 1575 
fulltime[0] 1.0
fulltime[1] 1.0
fulltime[2] 1.0
fulltime[3] -0.0
part_time[0] 5.0
part_time[1] 6.0
part_time[2] 9.0
part_time[3] 5.0
